In [ ]:
import os

folder_path = 'hci/references'
file_paths = []
# Iterate through all files in the folder
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        # Get the absolute path of the file
        file_path = os.path.join(root, file_name)
        # Append the file path to the list
        file_paths.append(file_path)

In [5]:
# !pip install -U sentence-transformers
import pandas as pd
# from sentence_transformers import SentenceTransformer, util

# model = SentenceTransformer("all-MiniLM-L6-v2")
gen_cq_file = 'ground-truth-cqs.txt'
ground_truth_cq_file = 'gpt-output/gpt-4o-temp-1.0.txt'
with open(ground_truth_cq_file, 'r') as f:
    expertCQs = [line.strip().split('. ')[1] for line in f]
with open(gen_cq_file, 'r') as f:
    genCQs = [line.strip().split('. ')[1] for line in f]




['What is an interactive computer system?', 'What is an interactive software system?', 'What is a complex interactive computer system?', 'What does make up the user interface of an interactive computer system?', 'What is a User?', 'How can a user interact in human–computer interactions?', 'Considering intentionality, how can a user interact with an interactive computer system?', 'Why does a user intentionally interact with an interactive computer system?', 'What does make up a complex user participation?', 'What is a human–computer interaction?', 'Considering the human–computer interaction, how can a user participation cause another user participation?', 'How is a user input processed by an interactive computer system?', 'How does a user receive an output from an interactive computer system?', 'How is a user input processed by an interactive computer system and how is the corresponding output presented to him/her?', 'How does a user evaluate if his/her goal was achieved in a human–comp

In [ ]:
data = pd.read_csv('/Users/sherry/python-coding/Prompting/requirement-engeneering/genCQs-expertCQs-reference.csv', names=['expertCQs', 'genCQs'], header=0)

genCQs = list(data['genCQs'])
expertCQs = list(data['expertCQs'])

genCQs_embeddings = model.encode(genCQs)
expertCQs_embeddings = model.encode(expertCQs)

# Compute cosine similarity between all pairs
cos_sim = util.cos_sim(genCQs_embeddings, expertCQs_embeddings)

# Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for genCQs_idx in range(77):
    for expertCQs_idx in range(77):
        all_sentence_combinations.append([cos_sim[genCQs_idx][expertCQs_idx], genCQs_idx, expertCQs_idx])

# Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top-5 most similar pairs:")
for score, genCQs_idx, expertCQs_idx in all_sentence_combinations[0:5]:
    print("{} \t {} \t {:.4f}".format(genCQs[genCQs_idx], expertCQs[expertCQs_idx], cos_sim[genCQs_idx][expertCQs_idx]))


# save cosine similarity score and the corresponding CQ pairs to a csv file
import torch

genCQ_ls = []
expertCQ_ls = []
score_ls = []
n = len(all_sentence_combinations)
for score, genCQs_idx, expertCQs_idx in all_sentence_combinations[0:n]:
    # print("{} \t {} \t {:.4f}".format(genCQs[genCQs_idx], expertCQs[expertCQs_idx], cos_sim[genCQs_idx][expertCQs_idx]))
    # print(f"{score.item():.4f}")
    # print(genCQs[genCQs_idx])
    # print(expertCQs[expertCQs_idx])
    genCQ_ls.append(genCQs[genCQs_idx])
    expertCQ_ls.append(expertCQs[expertCQs_idx])
    score_ls.append(f"{score.item():.4f}")
cos_sim_df = pd.DataFrame()
cos_sim_df['genCQ'] = genCQ_ls
cos_sim_df['expertCQ'] = expertCQ_ls
cos_sim_df['cos_score'] = score_ls

cos_sim_df.to_csv("/Users/sherry/python-coding/Prompting/requirement-engeneering/gen-expert-CQs-cos-one-reference.csv")